In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from IPython.display import clear_output

!pip install transformers
!pip install sentence-transformers
!pip install pyprind

clear_output()

In [3]:
%cd /content
!curl -c /tmp/cookies "https://drive.google.com/uc?export=download&id=1hbzc_P1FuxMkcabkgn9ZKinBwW683j45" > /tmp/intermezzo.html
!curl -L -b /tmp/cookies "https://drive.google.com$(cat /tmp/intermezzo.html | grep -Po 'uc-download-link" [^>]* href="\K[^"]*' | sed 's/\&amp;/\&/g')" > CUB_200_2011.tgz
!tar -xzvf /content/CUB_200_2011.tgz

!gdown --id 0B3y_msrWZaXLT1BZdVdycDY5TEE
!unzip /content/birds.zip

!mkdir /content/stage_1_images/

clear_output()

In [4]:
import pandas as pd
import numpy as np

import random
import os
import time
import gc
import pickle
from glob import glob
import pyprind

from PIL import Image
import PIL
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from torchvision.utils import save_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

from sentence_transformers import SentenceTransformer, util

---

In [5]:
class CUB_Dataset_1(Dataset):
    def __init__(self, root_dir="/content/", mode = 'train', transform = True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform
        self.bbox = self.load_bbox()     
        self._init_dataset()
        self.filenames = self.load_filenames()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.text_files = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, "CUB_200_2011/images")))
        if self.mode == 'train': 
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, "CUB_200_2011/images", dirs[dir], '*.jpg')))         
                self.files += files
                text_file = sorted(glob(os.path.join(self.data_dir, "birds/text_c10", dirs[dir], '*.txt')))
                self.text_files += text_file

    def load_bbox(self):
        bbox_path = os.path.join(self.data_dir , "CUB_200_2011/bounding_boxes.txt")
        df_bounding_boxes = pd.read_csv(bbox_path,
                                        delim_whitespace=True,
                                        header=None).astype(int)
        filepath = os.path.join(self.data_dir, "CUB_200_2011/images.txt")
        df_filenames = pd.read_csv(filepath, delim_whitespace=True, header=None)
        filenames = df_filenames[1].tolist()
        filename_bbox = {img_file[:-4]: [] for img_file in filenames}
        numImgs = len(filenames)
        for i in range(0, numImgs):
            # bbox = [x-left, y-top, width, height]
            bbox = df_bounding_boxes.iloc[i][1:].tolist()
            key = filenames[i][:-4]
            filename_bbox[key] = bbox
        return filename_bbox

    def get_img(self, img_path, bbox):
        img = Image.open(img_path).convert('RGB')
        width, height = img.size
        if bbox is not None:
            R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
            center_x = int((2 * bbox[0] + bbox[2]) / 2)
            center_y = int((2 * bbox[1] + bbox[3]) / 2)
            y1 = np.maximum(0, center_y - R)
            y2 = np.minimum(height, center_y + R)
            x1 = np.maximum(0, center_x - R)
            x2 = np.minimum(width, center_x + R)
            img = img.crop([x1, y1, x2, y2])
        load_size = int(64 * 76 / 64)
        img = img.resize((load_size, load_size), PIL.Image.BILINEAR)
        if self.transform is not None:
            img = self.transform(img)
        return img

    def load_filenames(self):
        filepath = os.path.join(self.data_dir, "birds/train/filenames.pickle")
        with open(filepath, 'rb') as f:
            filenames1 = pickle.load(f)
        filepath = os.path.join(self.data_dir, "birds/test/filenames.pickle")
        with open(filepath, 'rb') as f:
            filenames2 = pickle.load(f)
        filenames = filenames1 + filenames2
        return filenames

    def _init_transform(self):
        self.transform = transforms.Compose([
            transforms.RandomCrop((64, 64)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __getitem__(self, index):
        key = self.filenames[index]
        bbox = self.bbox[key]
        img = self.get_img(self.files[index], bbox)
        text = self.text_files[index]
        return img, text

    def __len__(self):
        return len(self.filenames)

In [6]:
class CUB_Dataset_2(Dataset):
    def __init__(self, root_dir, mode = 'train', transform = True):
        self.data_dir = root_dir
        self.mode = mode
        self.transforms = transform
        self.bbox = self.load_bbox()     
        self._init_dataset()
        self.filenames = self.load_filenames()
        if transform:
            self._init_transform()

    def _init_dataset(self):
        self.files = []
        self.text_files = []
        self.stage1_img_files = []
        dirs = sorted(os.listdir(os.path.join(self.data_dir, "CUB_200_2011/images")))
        if self.mode == 'train': 
            for dir in range(len(dirs)):
                files = sorted(glob(os.path.join(self.data_dir, "CUB_200_2011/images", dirs[dir], '*.jpg')))         
                self.files += files
                text_file = sorted(glob(os.path.join(self.data_dir, "birds/text_c10", dirs[dir], '*.txt')))
                self.text_files += text_file
                stage_1_files = sorted(glob(os.path.join(self.data_dir, "/stage_1_images", dirs[dir], '*.png')))
                self.stage1_img_files += stage_1_files

    def load_bbox(self):
        bbox_path = os.path.join(self.data_dir, "CUB_200_2011/bounding_boxes.txt")
        df_bounding_boxes = pd.read_csv(bbox_path, delim_whitespace=True, header=None).astype(int)
        filepath = os.path.join(self.data_dir, "CUB_200_2011/images.txt")
        df_filenames = pd.read_csv(filepath, delim_whitespace=True, header=None)
        filenames = df_filenames[1].tolist()
        filename_bbox = {img_file[:-4]: [] for img_file in filenames}
        numImgs = len(filenames)
        for i in range(0, numImgs):
            # bbox = [x-left, y-top, width, height]
            bbox = df_bounding_boxes.iloc[i][1:].tolist()
            key = filenames[i][:-4]
            filename_bbox[key] = bbox
        return filename_bbox

    def get_img(self, img_path, bbox):
        img = Image.open(img_path).convert('RGB')
        width, height = img.size
        if bbox is not None:
            R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
            center_x = int((2 * bbox[0] + bbox[2]) / 2)
            center_y = int((2 * bbox[1] + bbox[3]) / 2)
            y1 = np.maximum(0, center_y - R)
            y2 = np.minimum(height, center_y + R)
            x1 = np.maximum(0, center_x - R)
            x2 = np.minimum(width, center_x + R)
            img = img.crop([x1, y1, x2, y2])
        load_size = int(256 * 76 / 64)
        img = img.resize((load_size, load_size), PIL.Image.BILINEAR)
        if self.transform is not None:
            img = self.transform(img)
        return img

    def load_filenames(self):
        filepath = os.path.join(self.data_dir, "birds/train/filenames.pickle")
        with open(filepath, 'rb') as f:
            filenames1 = pickle.load(f)
        filepath = os.path.join(self.data_dir, "birds/test/filenames.pickle")
        with open(filepath, 'rb') as f:
            filenames2 = pickle.load(f)
        filenames = filenames1 + filenames2
        return filenames

    def _init_transform(self):
        self.transform1 = transforms.Compose([
            #transforms.RandomCrop((64, 64)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
        self.transform = transforms.Compose([
            transforms.RandomCrop((256, 256)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])
    
    def __getitem__(self, index):
        key = self.filenames[index]
        bbox = self.bbox[key]
        img = self.get_img(self.files[index], bbox)
        text = self.text_files[index]
        stageI_img = Image.open(self.stage1_img_files[index]).convert('RGB')
        stageI_img = self.transform1(stageI_img)
        return stageI_img, img, text

    def __len__(self):
        return len(self.stage1_img_files)

---

In [7]:
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

---

In [8]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)
    elif classname.find('Linear') != -1:
        m.weight.data.normal_(0.0, 0.02)
        if m.bias is not None:
            m.bias.data.fill_(0.0)

In [9]:
class Conditioning_Augmentation_StageI(nn.Module):
    def __init__(self):
        super(Conditioning_Augmentation_StageI, self).__init__()

        self.fc1 = nn.Linear(768, 256)
        self.relu = nn.ReLU()
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    def forward(self, x):
        
        x = x.to(device)
        y = self.relu(self.fc1(x))
        u0 = y[:, :128]
        logvar = y[:, 128:]
        sigma0 = torch.exp(logvar/2)
        epsilon = torch.randn((x.shape[0], 128)).to(self.device)
        out = u0 + sigma0*epsilon
        return out, u0, logvar

class Conditioning_Augmentation_StageII(nn.Module):
    def __init__(self):
        super(Conditioning_Augmentation_StageII, self).__init__()

        self.fc1 = nn.Linear(768, 256)
        self.relu = nn.ReLU()

    def forward(self, x):
        
        x = x.to(device)
        #print(x.shape)
        y = self.relu(self.fc1(x))
        u0 = y[:, :128]
        logvar = y[:, 128:]
        sigma0 = torch.exp(logvar/2)
        epsilon = torch.randn((x.shape[0], 128)).to(device)
        out = u0 + sigma0*epsilon
        return out, u0, logvar

In [10]:
class DownSample1(nn.Module):
    def __init__(self):
        super(DownSample1, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=4, stride=2, padding=1, bias = False)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias = False)
        self.batchnorm3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias = False)
        self.batchnorm4 = nn.BatchNorm2d(512)

        self.leakyrelu = nn.LeakyReLU(0.2, inplace = True)

    def forward(self, x):
        
        x = x.to(device)
        x = self.leakyrelu(self.conv1(x))
        x = self.leakyrelu(self.batchnorm2(self.conv2(x)))
        x = self.leakyrelu(self.batchnorm3(self.conv3(x)))
        x = self.leakyrelu(self.batchnorm4(self.conv4(x)))

        return x

class DownSample2(nn.Module):
    def __init__(self):
        super(DownSample2, self).__init__()

        self.conv1 = nn.Conv2d(3, 128, kernel_size=3, stride=1, padding=1, bias = False)
        
        self.conv2 = nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(256)

        self.conv3 = nn.Conv2d(256, 512, kernel_size=4, stride=2, padding=1, bias = False)
        self.batchnorm3 = nn.BatchNorm2d(512)

        self.relu = nn.ReLU(inplace=True)

    def forward(self, x):

        x = x.to(device)
        x = self.relu(self.conv1(x))
        x = self.relu(self.batchnorm2(self.conv2(x)))
        x = self.relu(self.batchnorm3(self.conv3(x)))

        return x

class DownSample3(nn.Module):
    def __init__(self):
        super(DownSample3, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size = 4, stride = 2, padding = 1, bias = False)
        
        self.conv2 = nn.Conv2d(64, 128, kernel_size = 4, stride = 2, padding = 1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 256, kernel_size = 4, stride = 2, padding = 1, bias = False)
        self.batchnorm3 = nn.BatchNorm2d(256)

        self.conv4 = nn.Conv2d(256, 512, kernel_size = 4, stride = 2, padding = 1, bias = False)
        self.batchnorm4 = nn.BatchNorm2d(512)

        self.conv5 = nn.Conv2d(512, 1024, kernel_size = 4, stride = 2, padding = 1, bias = False)
        self.batchnorm5 = nn.BatchNorm2d(1024)

        self.conv6 = nn.Conv2d(1024, 2048, kernel_size = 4, stride = 2, padding = 1, bias = False)
        self.batchnorm6 = nn.BatchNorm2d(2048)

        self.conv7 = nn.Conv2d(2048, 1024, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.batchnorm7 = nn.BatchNorm2d(1024)

        self.conv8 = nn.Conv2d(1024, 512, kernel_size = 3, stride = 1, padding = 1, bias = False)
        self.batchnorm8 = nn.BatchNorm2d(512)
        self.leakyrelu = nn.LeakyReLU(0.2)

    def forward(self, x):

        x = x.to(device)
        x = self.leakyrelu(self.conv1(x))
        x = self.leakyrelu(self.batchnorm2(self.conv2(x)))
        x = self.leakyrelu(self.batchnorm3(self.conv3(x)))
        x = self.leakyrelu(self.batchnorm4(self.conv4(x)))
        x = self.leakyrelu(self.batchnorm5(self.conv5(x)))
        x = self.leakyrelu(self.batchnorm6(self.conv6(x)))
        x = self.leakyrelu(self.batchnorm7(self.conv7(x)))
        x = self.leakyrelu(self.batchnorm8(self.conv8(x)))

        return x

In [11]:
class Generator_StageI(nn.Module):
    def __init__(self, condaug1):
        super(Generator_StageI, self).__init__()

        # In: [batch_size, 128]
        self.CA1 = condaug1()

        self.fc = nn.Sequential(
            nn.Linear(228, 4*4*128*8),
            nn.BatchNorm1d(4*4*128*8),
            nn.ReLU(True))
        
        self.upsample1 = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv1 = nn.Conv2d(128*8, 64*8, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(64*8)

        self.upsample2 = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv2 = nn.Conv2d(64*8, 32*8, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(32*8)

        self.upsample3 = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv3 = nn.Conv2d(32*8, 16*8, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm3 = nn.BatchNorm2d(16*8)

        self.upsample4 = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv4 = nn.Conv2d(16*8, 8*8, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm4 = nn.BatchNorm2d(8*8)

        self.conv5 = nn.Conv2d(8*8, 3, kernel_size=3, stride=1, padding=1, bias = False)
        self.tanh = nn.Tanh()
        self.relu = nn.ReLU()

    def forward(self, x):

        x = x.to(device)
        x, u0, logvar = self.CA1(x)
        z = torch.randn((x.shape[0], 100)).to(device)
        x = torch.cat((x, z), 1)
        x = self.fc(x)
        x = torch.reshape(x, (-1, 128*8, 4, 4))
        x = self.relu(self.batchnorm1(self.conv1(self.upsample1(x))))
        x = self.relu(self.batchnorm2(self.conv2(self.upsample2(x))))
        x = self.relu(self.batchnorm3(self.conv3(self.upsample3(x))))
        x = self.relu(self.batchnorm4(self.conv4(self.upsample4(x))))
        x = self.tanh(self.conv5(x))

        return x, u0, logvar

In [12]:
class Discriminator_StageI(nn.Module):
    def __init__(self, downsample):
        super(Discriminator_StageI, self).__init__()

        self.fc1 = nn.Linear(768, 128)
        self.downsample = downsample()
        self.conv1 = nn.Conv2d(640, 512, kernel_size=1, stride=1, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(512)
        self.leakyrelu = nn.LeakyReLU(0.2)

        self.conv2 = nn.Conv2d(512, 1, kernel_size = 4, stride = 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, text):
        
        x = x.to(device)
        text = text.to(device)
        x = self.downsample(x)   
        text = self.fc1(text)
        text = text.unsqueeze(2)
        text = text.unsqueeze(3)
        text1 = torch.cat((text, text, text, text), 2)
        text = torch.cat((text1, text1, text1, text1), 3)
        x = torch.cat((x, text), 1)
        x = self.leakyrelu(self.batchnorm1(self.conv1(x))) 
        x = self.conv2(x)
        x = torch.squeeze(x, 3)
        x = torch.squeeze(x, 2)
        x = self.sigmoid(x)

        return x

In [13]:
class ResidualBlock(nn.Module):
    def __init__(self):
        super(ResidualBlock, self).__init__()

        self.conv1 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(512)

        self.conv2 = nn.Conv2d(512, 512, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(512)

        self.relu = nn.ReLU()

    def forward(self, x):

        x = x.to(device)
        identity = x
        x = self.relu(self.batchnorm1(self.conv1(x)))
        x = self.batchnorm2(self.conv2(x))
        x = x + identity
        x = self.relu(x)

        return x

In [14]:
class UpSampling2(nn.Module):
    def __init__(self):
        super(UpSampling2, self).__init__()

        self.upsample = nn.Upsample(scale_factor=2, mode='nearest')
        self.conv1 = nn.Conv2d(512, 256, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(256)

        self.conv2 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm2 = nn.BatchNorm2d(128)

        self.conv3 = nn.Conv2d(128, 64, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm3 = nn.BatchNorm2d(64)

        self.conv4 = nn.Conv2d(64, 32, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm4 = nn.BatchNorm2d(32)

        self.conv5 = nn.Conv2d(32, 3, kernel_size=3, stride=1, padding=1, bias = False)
        self.relu = nn.ReLU()

    def forward(self, x):

        x = x.to(device)
        x = self.relu(self.batchnorm1(self.conv1(self.upsample(x))))
        x = self.relu(self.batchnorm2(self.conv2(self.upsample(x))))
        x = self.relu(self.batchnorm3(self.conv3(self.upsample(x))))
        x = self.relu(self.batchnorm4(self.conv4(self.upsample(x))))
        x = self.conv5(x)

        return x

In [15]:
class Generator_StageII(nn.Module):
    def __init__(self, downsample, resblock, upsample, condaug2):
        super(Generator_StageII, self).__init__()

        self.downsample = downsample()
        self.resblock = resblock()
        self.upsample = upsample()
        self.CA2 = condaug2()
        self.conv = nn.Conv2d(640, 512, kernel_size=3, stride=1, padding=1, bias = False)
        self.batchnorm = nn.BatchNorm2d(512)
        self.relu = nn.ReLU(True)
        self.tanh = nn.Tanh()
    def forward(self, x, text):

        x = x.to(device)
        text = text.to(device)
        text, u0, logvar = self.CA2(text)
        text = text.unsqueeze(2)
        text = text.unsqueeze(3)
        text = text.repeat(1, 1, 16, 16)
        x = self.downsample(x)
        x = torch.cat((x, text), 1)
        x = self.relu(self.batchnorm(self.conv(x)))
        x = self.resblock(self.resblock(self.resblock(self.resblock(x))))
        x = self.upsample(x)
        x = self.tanh(x)

        return x, u0, logvar

In [16]:
class Discriminator_StageII(nn.Module):
    def __init__(self, downsample):
        super(Discriminator_StageII, self).__init__()
        
        self.fc0 = nn.Linear(768, 128)
        self.downsample = downsample()
        self.conv1 = nn.Conv2d(640, 512, kernel_size=3, stride=1, padding = 1, bias = False)
        self.batchnorm1 = nn.BatchNorm2d(512)
        self.leakyrelu = nn.LeakyReLU(0.2)
        self.conv2 = nn.Conv2d(512, 1, kernel_size = 4, stride = 4)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, text):
        
        x = x.to(device)
        text = text.to(device)
        x = self.downsample(x)  
        text = self.fc0(text)
        text = text.unsqueeze(2)
        text = text.unsqueeze(3)
        text = text.repeat(1, 1, 4, 4)
        x = torch.cat((x, text), 1)
        x = self.leakyrelu(self.batchnorm1(self.conv1(x))) 
        x = self.sigmoid(self.conv2(x))  
        x = x.squeeze(3)
        x = x.squeeze(2)

        return x

In [17]:
net_g_stage_1 = Generator_StageI(Conditioning_Augmentation_StageI).to(device)
net_g_stage_1 = net_g_stage_1.apply(weights_init)

net_d_stage_1 = Discriminator_StageI(DownSample1).to(device)
net_d_stage_1 = net_d_stage_1.apply(weights_init)

net_g_stage_2 = Generator_StageII(DownSample2, ResidualBlock, UpSampling2, Conditioning_Augmentation_StageII).to(device)
net_g_stage_2 = net_g_stage_2.apply(weights_init)

net_d_stage_2 = Discriminator_StageII(DownSample3).to(device)
net_d_stage_2 = net_d_stage_2.apply(weights_init)

sbert_model = SentenceTransformer('paraphrase-mpnet-base-v2')

clear_output()

---

In [18]:
criterionBCE = nn.BCELoss().to(device)

In [19]:
def KL_loss(mu, logvar):
    KLD_element = mu.pow(2).add_(logvar.exp()).mul_(-1).add_(1).add_(logvar)
    KLD = torch.mean(KLD_element).mul_(-0.5)
    return KLD

In [20]:
def train_net_d_stage_1(real_images, wrong_images, text, generator, discriminator, optimizer, criterion, device):

    optimizer.zero_grad()

    real_images = real_images.to(device)
    text = text.to(device)
    real_pred = discriminator(real_images, text)
    real_targets = torch.ones(real_images.size(0),1).to(device)
    real_loss = criterion(real_pred, real_targets)
    real_score = torch.mean(real_pred).item()

    fake_images, mu, logvar = generator(text)
    fake_pred1 = discriminator(fake_images, text)
    fake_targets1 = torch.zeros(fake_images.size(0),1).to(device)
    fake_loss1 = criterion(fake_pred1, fake_targets1)
    fake_score1 = torch.mean(fake_pred1).item()

    wrong_images = wrong_images.to(device)
    fake_pred2 = discriminator(wrong_images, text)
    fake_targets2 = torch.zeros(wrong_images.size(0),1).to(device)
    fake_loss2 = criterion(fake_pred2, fake_targets2)
    fake_score2 = torch.mean(fake_pred2).item()

    discriminator_loss = real_loss + (fake_loss1 + fake_loss2)/2
    discriminator_loss.backward()
    optimizer.step()

    return discriminator_loss.item(), real_score, (fake_score1+fake_score2)/2

In [21]:
def train_net_g_stage_1(text, generator, discriminator, optimizer, criterion, criterion_KL, device):

    optimizer.zero_grad()

    text = text.to(device)
    generator_images, mu, logvar = generator(text)
    generator_pred = discriminator(generator_images, text)
    generator_targets = torch.ones(batch_size, 1).to(device)

    gen_bin_loss = criterion(generator_pred, generator_targets)
    generator_score = torch.mean(generator_pred).item()
    kl_loss = criterion_KL(mu, logvar)

    generator_loss = gen_bin_loss + 2*kl_loss
    generator_loss.backward()
    optimizer.step()

    return generator_loss.item(), gen_bin_loss.item() , kl_loss.item(), generator_score

In [22]:
def train_net_d_stage_2(real_images, wrong_images, stage_1_img, text, generator, discriminator, optimizer, criterion, device):

    optimizer.zero_grad()

    real_images = real_images.to(device)
    text = text.to(device)
    real_pred = discriminator(real_images, text)
    real_targets = torch.ones(real_images.size(0),1).to(device)
    real_loss = criterion(real_pred, real_targets)
    real_score = torch.mean(real_pred).item()

    fake_images, mu, logvar = generator(stageI_img, text)
    fake_pred1 = discriminator(fake_images, text)
    fake_targets1 = torch.zeros(fake_images.size(0),1).to(device)
    fake_loss1 = criterion(fake_pred1, fake_targets1)
    fake_score1 = torch.mean(fake_pred1).item()

    wrong_images = wrong_images.to(device)
    fake_pred2 = discriminator(wrong_images, text)
    fake_targets2 = torch.zeros(wrong_images.size(0),1).to(device)
    fake_loss2 = criterion(fake_pred2, fake_targets2)
    fake_score2 = torch.mean(fake_pred2).item()

    discriminator_loss = (fake_loss1 + fake_loss2)/2 + real_loss
    discriminator_loss.backward()
    optimizer.step()

    return discriminator_loss.item(), real_score, (fake_score1 + fake_score2)/2

In [23]:
def train_net_g_stage_2(stage_1_img, text, generator, discriminator, optimizer, criterion, criterion_KL, device):

    optimizer.zero_grad()
    
    stage_1_img = stage_1_img.to(device)
    text = text.to(device)
    generator_images, mu, logvar = generator(stage_1_img, text)
    generator_pred = discriminator(generator_images, text)
    generator_targets = torch.ones(batch_size, 1).to(device)
    gen_bin_loss = criterion(generator_pred, generator_targets)
    generator_score = torch.mean(generator_pred).item()
    kl_loss = criterion_KL(mu, logvar)

    generator_loss = gen_bin_loss + 2*kl_loss
    generator_loss.backward()
    optimizer.step()

    return generator_loss.item(), generator_score

---

In [24]:
# Change to your Drive Path

CHECKPOINT = "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/StackGAN/checkpoints"
IMAGES = "/content/drive/MyDrive/Projects/Clubs/Analytics/Coord Projects/Model Zoo/StackGAN/images"

In [25]:
def save_samples_stage_1(index1, text, show=True):
    fake_images, a, b = net_g_stage_1(text)
    fake_images = fake_images[0:16,:,:,:]
    fake_fname = 'generated-images-{}.png'.format(index1)
    save_image((fake_images), os.path.join(IMAGES, "stage_1", fake_fname), nrow=4)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

In [26]:
def save_samples_stage_2(index1, stageI_img, text, show=True):
    fake_images, a, b = net_g_stage_2(stageI_img, text)
    fake_images = fake_images[0:4,:,:,:]
    fake_fname = 'generated-images-{}.png'.format(index1)
    save_image((fake_images), os.path.join(IMAGES, "stage_2", fake_fname), nrow=2)
    if show:
        fig, ax = plt.subplots(figsize=(8, 8))
        ax.set_xticks([]); ax.set_yticks([])
        ax.imshow(make_grid(fake_images.cpu().detach(), nrow=8).permute(1, 2, 0))

---

In [27]:
epoch_d_stage_1_losses = []             
epoch_g_stage_1_losses = []
epoch_real_score_stage_1 = []
epoch_fake_score_stage_1 = []
epoch_generator_score_stage_1 = []

In [28]:
trainset_1 = CUB_Dataset_1(root_dir='/content/')
trainloader_1 = DataLoader(trainset_1, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True)

In [29]:
epochs = 500
lrG = 0.0002
lrD = 0.0002

optimizer_g_stage_1 = torch.optim.Adam(net_g_stage_1.parameters(), lr=lrG, betas=(0.5,0.999))
optimizer_d_stage_1 = torch.optim.Adam(net_d_stage_1.parameters(), lr=lrD, betas=(0.5,0.999))

In [ ]:
for epoch in range(1, epochs+1):
    
    print("Starting Epoch [{}/{}]".format(epoch, epochs))
    train_d_loss_batch = []
    train_g_loss_batch = []
    train_real_score = []
    train_fake_score = []
    train_generator_score = []

    net_g_stage_1.train()
    net_d_stage_1.train()

    bar = pyprind.ProgBar(len(trainloader_1), bar_char='█')
    for idx,(real_images, text) in enumerate(trainloader_1):

        text = list(text)
        embedding = []
        for i in range(len(text)):
            my_file = open(text[i], "r")
            content = my_file.read()
            embedding.append(content)
        embed = sbert_model.encode(embedding)
        embed = torch.from_numpy(embed)

        wrong_images = torch.flip(real_images, [0])
        discriminator_loss, real_score, fake_score = train_net_d_stage_1(real_images, wrong_images, embed, net_g_stage_1, net_d_stage_1, optimizer_d_stage_1, criterionBCE, device)
        generator_loss, a, b, generator_score = train_net_g_stage_1(embed, net_g_stage_1, net_d_stage_1, optimizer_g_stage_1, criterionBCE, KL_loss, device)
        train_d_loss_batch.append(discriminator_loss)
        train_g_loss_batch.append(generator_loss)
        train_real_score.append(real_score)
        train_fake_score.append(fake_score)
        train_generator_score.append(generator_score)

        if (idx+1)%180 == 0:
            emb_save = embed[0:16, :]
            save_samples_stage_1(epoch, emb_save, show=False)

        bar.update()
        torch.cuda.empty_cache()
        gc.collect()

    if (epoch+1)%50 == 0:
        lrG = lrG/2
        lrD = lrD/2
        print(f"Learning Rate Halved: {lrG} {lrD}")

    epoch_d_stage_1_losses.append(sum(train_d_loss_batch)/len(trainloader_1))
    epoch_g_stage_1_losses.append(sum(train_g_loss_batch)/len(trainloader_1))
    epoch_real_score_stage_1.append(sum(train_real_score)/len(trainloader_1))
    epoch_fake_score_stage_1.append(sum(train_fake_score)/len(trainloader_1))
    epoch_generator_score_stage_1.append(sum(train_generator_score)/len(trainloader_1))

    torch.save({
        'model_state_dict': net_d_stage_1.state_dict(),
        'optimizer_state_dict': optimizer_d_stage_1.state_dict(),
        }, os.path.join(CHECKPOINT, "net_d_stage_1.pth"))

    torch.save({
        'model_state_dict': net_g_stage_1.state_dict(),
        'optimizer_state_dict': net_g_stage_1.state_dict(),
        }, os.path.join(CHECKPOINT, "net_g_stage_1.pth"))

    print(f"Discriminator Loss: {epoch_d_stage_1_losses[-1]:.5f} | Generator Loss: {epoch_g_stage_1_losses[-1]:.5f}")
    print(f"Real Score: {epoch_real_score_stage_1[-1]:.5f} | Fake Score: {epoch_fake_score_stage_1[-1]:.5f} | Generator Score: {epoch_generator_score_stage_1[-1]:.5f}")

    torch.cuda.empty_cache()
    gc.collect()
    
    print("Finished Epoch [{}/{}]\n".format(epoch, epochs))

Starting Epoch [1/500]


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:59


Discriminator Loss: 1.34407 | Generator Loss: 2.72225
Real Score: 0.57904 | Fake Score: 0.42283 | Generator Score: 0.14368
Finished Epoch [1/500]

Starting Epoch [2/500]


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:01:57


Discriminator Loss: 1.26293 | Generator Loss: 1.59499
Real Score: 0.57737 | Fake Score: 0.41988 | Generator Score: 0.29517
Finished Epoch [2/500]

Starting Epoch [3/500]


...

0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:39


Discriminator Loss: 0.07068 | Generator Loss: 6.87718
Real Score: 0.97591 | Fake Score: 0.02518 | Generator Score: 0.02049
Finished Epoch [251/500]

Starting Epoch [252/500]


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:02:40


Discriminator Loss: 0.11800 | Generator Loss: 6.77108
Real Score: 0.96297 | Fake Score: 0.03666 | Generator Score: 0.02949
Finished Epoch [252/500]

Starting Epoch [253/500]


0% [███████████████████████████   ] 100% | ETA: 00:00:15

---

In [ ]:
checkpoint_net_g_stage_1 = torch.load(os.path.join(CHECKPOINT, "net_g_stage_1.pth"))
net_g_stage_1.load_state_dict(checkpoint_net_g_stage_1['model_state_dict'])
net_g_stage_1.to(device)

In [ ]:
net_g_stage_1.eval()

dirs = sorted(os.listdir(os.path.join("/content/", "CUB_200_2011/images")))
for dir in range(len(dirs)):
    try:
        os.mkdir(os.path.join("/content/stage_1_images/", dirs[dir]))
    except:
        pass
    try:
        os.mkdir(os.path.join(IMAGES, "inputs", dirs[dir]))
    except:
        pass

    images = sorted(glob(os.path.join("/content/", "CUB_200_2011/images", dirs[dir], '*.jpg')))         
    text_files = sorted(glob(os.path.join("/content/", "birds/text_c10", dirs[dir], '*.txt')))

    for text in text_files:
    
        fname = os.path.splitext(os.path.basename(text))[0] + ".png"
        my_file = open(text, "r")
        content = my_file.read()
        embed = sbert_model.encode(content)
        embed = torch.from_numpy(embed)
        embed = embed.unsqueeze(0)

        generator_images, mu, logvar = net_g_stage_1(embed)
        generator_images = generator_images.squeeze(0).detach().cpu().permute(1, 2, 0).numpy()
        result = Image.fromarray((generator_images * 255).astype(np.uint8))
        result.save(os.path.join("/content/stage_1_images/", dirs[dir], fname))
        result.save(os.path.join(IMAGES, "inputs", dirs[dir], fname))

        gc.collect()

---

In [ ]:
epoch_d_stage_2_losses = []             
epoch_g_stage_2_losses = []
epoch_real_score_stage_2 = []
epoch_fake_score_stage_2 = []
epoch_generator_score_stage_2 = []

In [ ]:
trainset_2 = CUB_Dataset_2(root_dir='/content/')
trainloader_2 = DataLoader(trainset_2, batch_size=batch_size, shuffle=True, num_workers=2, drop_last=True, pin_memory = True)

In [ ]:
epochs = 500
lrG = 0.0002
lrD = 0.0002

optimizer_g_stage_2 = torch.optim.Adam(net_g_stage_2.parameters(), lr=lrG, betas=(0.5,0.999))
optimizer_d_stage_2 = torch.optim.Adam(net_d_stage_2.parameters(), lr=lrD, betas=(0.5,0.999))

In [ ]:
for epoch in range(1, epochs+1):
    
    print("Starting Epoch [{}/{}]".format(epoch, epochs))
    train_d_loss_batch = []
    train_g_loss_batch = []
    train_real_score = []
    train_fake_score = []
    train_generator_score = []

    net_g_stage_2.train()
    net_d_stage_2.train()

    bar = pyprind.ProgBar(len(trainloader_2), bar_char='█')
    for idx,(stage_1_img, real_images, text) in enumerate(trainloader_2):
        text = list(text)
        embedding = []
        for i in range(len(text)):
            my_file = open(text[i], "r")
            content = my_file.read()
            embedding.append(content)
        embed = sbert_model.encode(embedding)
        embed = torch.from_numpy(embed)

        wrong_images = torch.flip(real_images, [0])
        discriminator_loss, real_score, fake_score = train_net_d_stage_2(real_images, wrong_images, stage_1_img, embed, net_g_stage_2, net_d_stage_2, optimizer_d_stage_2, criterionBCE, device)
        generator_loss, generator_score = train_net_g_stage_2(stage_1_img, embed, net_g_stage_2, net_d_stage_2, optimizer_g_stage_2, criterionBCE, KL_loss, device)
        train_d_loss_batch.append(discriminator_loss)
        train_g_loss_batch.append(generator_loss)
        train_real_score.append(real_score)
        train_fake_score.append(fake_score)
        train_generator_score.append(generator_score)

        if (idx+1)%180 == 0:
            emb_save = embed[0:4, :]
            stage_1_img_save = stage_1_img[0:4, :, :, :]
            save_samples_stage_2(epoch+372, stage_1_img_save, emb_save, show=False)

        bar.update()
        torch.cuda.empty_cache()
        gc.collect()

    if (epoch+1)%50 == 0:
        lrG = lrG/2
        lrD = lrD/2
        print(f"Learning Rate Halved: {lrG} {lrD}")

    epoch_d_stage_2_losses.append(sum(train_d_loss_batch)/len(trainloader_2))
    epoch_g_stage_2_losses.append(sum(train_g_loss_batch)/len(trainloader_2))
    epoch_real_score.append(sum(train_real_score)/len(trainloader_2))
    epoch_fake_score.append(sum(train_fake_score)/len(trainloader_2))
    epoch_generator_score.append(sum(train_generator_score)/len(trainloader_2))

    torch.save({
        'model_state_dict': net_g_stage_2.state_dict(),
        'optimizer_state_dict': optimizer_d_stage_2.state_dict(),
        }, os.path.join(CHECKPOINT, "net_d_stage_2.pth"))

    torch.save({
        'model_state_dict': net_g_stage_2.state_dict(),
        'optimizer_state_dict': optimizer_g_stage_2.state_dict(),
        }, os.path.join(CHECKPOINT, "net_g_stage_2.pth"))

    print(f"Discriminator Loss: {epoch_d_stage_2_losses[-1]:.5f} | Generator Loss: {epoch_g_stage_2_losses[-1]:.5f}")
    print(f"Real Score: {epoch_real_score_stage_2[-1]:.5f} | Fake Score: {epoch_fake_score_stage_2[-1]:.5f} | Generator Score: {epoch_generator_score_stage_2[-1]:.5f}")

    torch.cuda.empty_cache()
    gc.collect()

    print("Finished Epoch [{}/{}]\n".format(epoch, epochs))

---

In [ ]:
checkpoint_net_g_stage_1 = torch.load(os.path.join(CHECKPOINT, "net_g_stage_1.pth"))
net_g_stage_1.load_state_dict(checkpoint_net_g_stage_1['model_state_dict'])
net_g_stage_1.to(device)

checkpoint_net_g_stage_2 = torch.load(os.path.join(CHECKPOINT, "net_g_stage_2.pth"))
net_g_stage_2.load_state_dict(checkpoint_net_g_stage_2['model_state_dict'])
net_g_stage_2.to(device)

In [ ]:
net_g_stage_1.eval()
net_g_stage_2.eval()

embed = sbert_model.encode('''The bird is black in colour with white belly''')
embed = torch.from_numpy(embed)
embed = embed.unsqueeze(0)

generator_images, mu, logvar = net_g_stage_1(embed)
generator_images2, mu, logvar = net_g_stage_2(generator_images, embed)
generator_images2 = generator_images2.squeeze(0)

plt.imshow(generator_images2)